In [1]:
import numpy as np
import math 
from qiskit import *
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
from qiskit import IBMQ, Aer, execute
from qiskit.tools.visualization import plot_histogram
from qiskit.circuit.library import WeightedAdder
backend = BasicAer.get_backend('qasm_simulator')
shots = 1024
provider = IBMQ.load_account()

In [2]:
#Taking a list input which will be proccessed.
print("Please enter input list")
init=list(map(int,input().split()))

n=len(init)
#Calculating the no. of qubits required for qRAM address
adsize=math.ceil(math.log(n,2))

maxi=max(init)
#Calculating no. of qubits required for qRAM data
datsize=math.floor(math.log(maxi,2))+1

#Converting the input list to binary 
for i in range(n):
    init[i]=format(init[i], 'b').zfill(datsize)
    
#Creating quantum circuit
address = QuantumRegister(adsize)
data1 = QuantumRegister(datsize)
data2 = QuantumRegister(datsize)
oracle1= QuantumRegister(1)
cr = ClassicalRegister(adsize)
qc = QuantumCircuit(address,data1,data2,oracle1,cr)

#Initialising quantum circuit
qc.x(oracle1[0])
qc.h(oracle1[0])

qc.h(address[:])

#Creating oracle initial state
qc.h(data2[0])
for i in range(datsize-1):
    qc.x(data2[i+1])
    qc.cx(data2[i],data2[i+1])

#Calculating no. of iterations needed for optimal solution to grover's algorithm with 2 solutions present in the given list    
ite=round(np.pi/4*2**((adsize-1)/2)-0.5)

#running the circuit for ite. times as this will give better result 
for x in range(ite):
    
    #Storing data in qRAM
    for i in range(len(init)):
        dum=format(i, 'b').zfill(adsize)
        for j in range(len(dum)):
            if(dum[j]=='0'):
                qc.x(address[j])
        for k in range(len(init[i])):
            if(init[i][k]=='1'):
                qc.mct(address[:],data1[k])
        for j in range(len(dum)):
            if(dum[j]=='0'):
                qc.x(address[j])


    #Oracle    
    for i in range(datsize):
        qc.cx(data2[i],data1[i])
    
    #Phase flip for desired solution
    qc.mct(data1[:],oracle1[0])

    #Uncomputation of oracle
    for i in range(datsize):
        qc.cx(data2[i],data1[i])

        
    #Uncomputation of qRAM
    for i in range(len(init)):
        dum=format(i, 'b').zfill(adsize)
        for j in range(len(dum)):
            if(dum[j]=='0'):
                qc.x(address[j])
        for k in range(len(init[i])):
            if(init[i][k]=='1'):
                qc.mct(address[:],data1[k])
        for j in range(len(dum)):
            if(dum[j]=='0'):
                qc.x(address[j])

    #Diffusion of our solution
    qc.h(address[:])
    qc.x(address[:])
    qc.h(address[adsize-1])
    qc.mct(address[:adsize-1],address[adsize-1])
    qc.h(address[adsize-1])
    qc.x(address[:])
    qc.h(address[:])

#Ending of oracle
qc.h(oracle1[0])
qc.x(oracle1[0])

#Measuring the solution
qc.measure(address[:],cr[:])

#Reversing bits as per qiskit convention 
qc = qc.reverse_bits()

#measuring our solution using qasm_simulator
backend = Aer.get_backend('qasm_simulator')
job = execute(qc, backend=backend, shots=8000, seed_simulator=12345, backend_options={"fusion_enable":True})
result = job.result()
count =result.get_counts()

#printing result in ascending order 
{k: v for k, v in sorted(count.items(), key=lambda item: item[1])}


Please enter input list


 2 5 7 10


  job = backend.run(experiments, **run_kwargs)


{'01': 3997, '11': 4003}